In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2
import msmrd2.tools.quaternions as quats
import msmrd2.visualization as msmrdvis
from msmrd2.potentials import patchyProteinMAPK
from msmrd2.integrators import integratorMAPK 

## Units

### Boltzman constant
- $k_B = 1.38064852 \times 10^{-23} \frac{m^2}{s^2} \frac{kg}{K} \left(= \frac{nm^2}{ns^2}\frac{kg}{K}\right)$.

### Basic units
- Length (l): nanometers (nm)
- Energy ($\epsilon$) : $k_B T = kg \frac{nm^2}{ns^2}$
- Mass (m): kilogram (kg)

### Derived units
- Time: $l\sqrt{m/\epsilon} = nm$
- Temperature: $\epsilon/k_B =$ Kelvin ($K$)
- Force: $\epsilon/l = kg \frac{nm}{ns^2}$

### Reduced quantities (dimensionless)
- Reduced pair potential: $U^* = U/\epsilon$
- Reduced force: $F^* = F l/\epsilon$
- Reduced distance: $r^* = r/l$
- Reduced density: $\rho^*=\rho l^3$
- Reduced Temperature: $T^* = k_B T/\epsilon$
- Reduced Pressure: $P^* = Pl^3/\epsilon$

In [ ]:
# Define main variables and parameters
numMAPKs = 1
numKinases = 1 #3
numPhosphatases = 0 #2
numparticles = numMAPKs + numKinases + numPhosphatases
sigma = 5.0 #(nanometers) particles diameters
boxsize = 15 #50 #(nanometers) #2.3
D = 1.0E-3 #(nm^2/ns) Note 1.0E-3 nm^2/ns = 1 micrometer^2/s
Drot = 1.6E-4 #(rad^2/ns) Note 1.6E-4 rad^2/ns = 1.6E5 rad^2/s
kB = 1.38064852E-23 #(Boltzmann constant (nm^2/ns^2 * kg/K))
Temp = 300 #(Kelvin)
# For computations, we assume kBT=1, thus the force F must be: F=kBT f, where f is the force computed 
# from the potential. This means the plotted potential is on reduced units (not the distances though);
kBT = 1 

# Define arbitrarily large list of patchy particles
np.random.seed(seed=1) 
pyPartlist = [] 
anglePatches = np.pi/2
orientvector = np.array([np.cos(-anglePatches/2), np.sin(-anglePatches/2), 0.]) # same as patches coordinates 2

for i in range(numparticles):    
    overlap = True
    while overlap:
        position = np.array([boxsize*np.random.rand()-0.5*boxsize,
                             boxsize*np.random.rand()-0.5*boxsize,
                             boxsize*np.random.rand()-0.5*boxsize])
        overlap = False     
        for j in range(len(pyPartlist)):
            if np.linalg.norm(position - pyPartlist[j].position) < sigma:
                overlap = True
                continue
                
    orientation = np.array([np.random.rand(),np.random.rand(),np.random.rand(),np.random.rand()])
    orientation = orientation/np.linalg.norm(orientation)
    position = np.array([0,0,0]) + i*np.array([0.8*sigma,0,0])
    # ptype = i
    if i+1 <= numMAPKs:
        ptype = 0
    elif i+1 <= numMAPKs + numKinases:
        ptype = 1
    else: 
        ptype = 2
    part = msmrd2.particle(ptype, 0, D, Drot, position, orientation)
    if ptype == 1 or ptype == 2:
        rotOrientvector = msmrd2.tools.rotateVec(orientvector, orientation)
        part.setOrientVector(rotOrientvector)
    pyPartlist.append(part)

In [ ]:
# Alternative fixed position set up (for testing)
#position1 = np.array([0.0, 0.0, 0.0]);
#position2 = np.array([np.cos(np.pi/4), np.sin(-np.pi/4), 0]);
#position3 = np.array([1, 0, 0]);
#orientation1 = np.array([1.0, 0.0, 0.0, 0.0]);
#orientation2 = np.array([np.cos(np.pi/2), 0.0, 0.0, np.sin(np.pi/2)]);
#orientation3 = np.array([1.0, 0.0, 0.0, 0.0]);
#part1 = msmrd2.particle(0,0,D, Drot, position1, orientation1)
#part2 = msmrd2.particle(1,0,D, Drot, position2, orientation2)
#part3 = msmrd2.particle(2,0,D, Drot, position3, orientation3)
#pyPartlist = [part1, part2, part3] 

In [ ]:
# Create list of particles that can be read from msmrd
# note the particles in this list will be independent from the python list.
partlist = msmrd2.integrators.particleList(pyPartlist)

In [ ]:
# Over-damped Langevin integrator definition
dt = 0.1 # 0.001 (nanoseconds)
seed = -1 #9 # seed = -1 uses random device as seed
bodytype = 'rigidbody' # three rotational degrees of freedom
trel = 1000 # (nanoseconds) Note 1000 ns = 1 micro second. Can be varied up to 10 miliseconds
reactivationRateK = np.log(2)/trel 
reactivationRateP = np.log(2)/trel
mapkIndex = [0]
kinaseIndex = [1]
phosIndex = [2]
integrator = integratorMAPK(dt, seed, bodytype, anglePatches, reactivationRateK, reactivationRateP, sigma,
                            mapkIndex, kinaseIndex, phosIndex) 
integrator.setKbT(kBT) # Note we use 1 for computations of KbT

In [ ]:
# Define boundary (choose either spherical or box)
boxBoundary = msmrd2.box(boxsize,boxsize,boxsize,'reflective')
integrator.setBoundary(boxBoundary)
#integrator.disableDeactivation()

In [ ]:
# Define MAPK potntial based on patchy particles
strength = 100 #60 #65
# This values are fixed and should match those used to determine metastable states in potential and trajectory.
patchesCoordinates1 = [np.array([np.cos(anglePatches/2), np.sin(anglePatches/2), 0.]), 
                       np.array([np.cos(-anglePatches/2), np.sin(-anglePatches/2), 0.])]
patchesCoordinates2 = [ np.array([np.cos(-anglePatches/2), np.sin(-anglePatches/2), 0.]) ]
# Defines patchy protein potential
potentialPatchyProteinMAPK = patchyProteinMAPK(sigma, strength, patchesCoordinates1, patchesCoordinates2)
# Incorporate potential into integrator
integrator.setPairPotential(potentialPatchyProteinMAPK)

In [ ]:
# Integrate the particles, save to .xyz to produce VMD output (additional overhead)
timesteps = 10000000 #250000 #5000000 
stride = 250 #1 #250 #250 #1000
datafile  = open('../../../data/vmd/patchyProteinMAPK.xyz', 'w')
for i in range(timesteps):
    if i%stride == 0:
        datafile.write(str(5*numMAPKs + 2*(numKinases + numPhosphatases)) + '\n')
        datafile.write(str(0) + '\n')
    for j, part in enumerate(partlist):
        if i%stride == 0:
            if part.type == 0:
                v0 = part.position
                v1 = v0 + 0.5*sigma*quats.rotateVec(patchesCoordinates1[0], part.orientation)
                v2 = v0 + 0.5*sigma*quats.rotateVec(patchesCoordinates1[1], part.orientation)
                datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
                if (part.state == 0):
                    datafile.write('type_1' + ' ' + ' '.join(map(str, v1)) + '\n')
                    datafile.write('type_1' + ' ' + ' '.join(map(str, v2)) + '\n')
                    datafile.write('type_2' + ' ' + ' '.join(map(str, v0)) + '\n')
                    datafile.write('type_2' + ' ' + ' '.join(map(str, v0)) + '\n')
                elif (part.state == 1):
                    datafile.write('type_1' + ' ' + ' '.join(map(str, v0)) + '\n')
                    datafile.write('type_1' + ' ' + ' '.join(map(str, v2)) + '\n')
                    datafile.write('type_2' + ' ' + ' '.join(map(str, v1)) + '\n')
                    datafile.write('type_2' + ' ' + ' '.join(map(str, v0)) + '\n')
                elif (part.state == 2):
                    datafile.write('type_1' + ' ' + ' '.join(map(str, v1)) + '\n')
                    datafile.write('type_1' + ' ' + ' '.join(map(str, v0)) + '\n')
                    datafile.write('type_2' + ' ' + ' '.join(map(str, v0)) + '\n')
                    datafile.write('type_2' + ' ' + ' '.join(map(str, v2)) + '\n')
                else:
                    datafile.write('type_1' + ' ' + ' '.join(map(str, v0)) + '\n')
                    datafile.write('type_1' + ' ' + ' '.join(map(str, v0)) + '\n')
                    datafile.write('type_2' + ' ' + ' '.join(map(str, v1)) + '\n')
                    datafile.write('type_2' + ' ' + ' '.join(map(str, v2)) + '\n')
            if part.type == 1:
                v0 = part.position
                if part.state == 0:
                    v1 = v0 + 0.5*sigma*quats.rotateVec(patchesCoordinates2[0], part.orientation)
                else:
                    v1 = 1 * v0
                datafile.write('type_3' + ' ' + ' '.join(map(str, v0)) + '\n')
                datafile.write('type_2' + ' ' + ' '.join(map(str, v1)) + '\n')
            if part.type == 2:
                v0 = part.position
                if part.state == 0:
                    v1 = v0 + 0.5*sigma*quats.rotateVec(patchesCoordinates2[0], part.orientation)
                else:
                    v1 = 1 * v0
                datafile.write('type_4' + ' ' + ' '.join(map(str, v0)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v1)) + '\n')
    integrator.integrate(partlist)
    if i%(50*stride) == 0:
        print("Percentage complete: ", 100*i/timesteps, "%  ", end="\r")
datafile.close()
# Generate TCL script to visualize with VMD
msmrdvis.generateTCL_patchyProteinMAPK(numparticles = numparticles, 
                                    outfname = "patchyProteinMAPK", 
                                    tclfname = "../../../data/vmd/patchyProteinMAPK_2vmd.tcl")
print("Percentage complete: ", 100, " %")

To load the movie go to /data/vmd and run in a terminal "vmd -e patchyProteinMAPK_2vmd.tcl".

## In case VMD is not desired

In case VMD output is not desired, below we simply integrate the model and show the output directly.

In [ ]:
## Integrate particle list and print only positions of first and last particle in list
#timesteps = 100
#print('{:<10s}{:<15s}{:<40s}{:<40s}'.format("Iteration", "Time", "Position 1", "Position 2"))
#for i in range(timesteps):
#    print('{:<10d}{:<15f}{:<40s}{:<50s}'.format(i, integrator.clock, str(partlist[0].position), str(partlist[1].position)))
#    print('{:<10d}{:<15f}{:<40s}{:<50s}'.format(i, integrator.clock, str(partlist[0].state), str(partlist[1].state)))

#    integrator.integrate(partlist)

# Calculate potential and plot

Plots the potential for the case when bidning sites i3 and i4 are aligned in one axis

In [ ]:
# Define MAPK potntial based on patchy particles
sigma = 5.0
strength = 10
maxdistance = 5*sigma
anglePatches = np.pi/2
# This values are fixed and should match those used to determine metastable states in potential and trajectory.
patchesCoordinates1 = [np.array([np.cos(anglePatches/2), np.sin(anglePatches/2), 0.]), 
                       np.array([np.cos(-anglePatches/2), np.sin(-anglePatches/2), 0.])]
patchesCoordinates2 = [ np.array([np.cos(-anglePatches/2), np.sin(-anglePatches/2), 0.]) ]

# Calculate potential for aligned molecules interaction between MAPK and kinase (same for phosphatase)
potentialPatchyProteinMAPKalt = patchyProteinMAPK(sigma, strength, patchesCoordinates1, patchesCoordinates2)

def patchyProteinMAPKevaluate(p1,p2,or1,or2):
    part1 = msmrd2.particle(0, 0, 1.0, 1.0, p1, or1)
    part2 = msmrd2.particle(1, 0, 1.0, 1.0, p2, or2)
    return potentialPatchyProteinMAPKalt.evaluate(part1, part2)

dist = np.arange(0,maxdistance, 0.01)
pos1 = np.array([0, 0., 0.])
pos2 = np.array([[i, 0., 0.] for i in dist])

# Orientation of particle 1 to align binding site to x-axis 
axisAngle = np.array([0,0,anglePatches/2]) 
orientation1 = quats.angle2quat(axisAngle)

# Orientation of particle 2 to align binding site to x-axis (facing opposite direction) 
axisAngle2 = np.array([0,0,anglePatches/2 - np.pi]) 
orientation2 = quats.angle2quat(axisAngle2)

potentialAligned = [patchyProteinMAPKevaluate(pos1,p2,orientation1,orientation2) for p2 in pos2]

In [ ]:
# Plot potential
plt.figure(figsize=(7,5))
fsize = 22
plt.rcParams.update({'font.size': fsize})
plt.plot(dist, potentialAligned, label = r'aligned MAPK and kinase', lw =2)
plt.plot(dist, 0*dist, '--k', lw=0.5)
#plt.legend(fancybox=True)
plt.legend(fontsize=28, labelspacing=2, framealpha=1.0, edgecolor='white', fancybox=True, bbox_to_anchor=(1.3, 1.0))
plt.xlim([0,maxdistance])
plt.xlabel(r'Relative distance')
plt.ylabel(r'Potential/$k_B T$ (reduced)')
#plt.yticks([])
#plt.savefig('patchyParticles_potential.pdf', bbox_inches='tight')